In [1]:
from trl import RewardTrainer
import pandas as pd

In [2]:
data = pd.read_csv("df1_train.csv")

In [3]:
data['prompt'] = data["wrong_text"].map(lambda x: f"Paraphrase logically: '{x}'")

In [4]:
data

,wrong_text,correct_text,prompt
0,the man is to the left of the shirt,the shirt is to the left of the man,Paraphrase logically: 'the man is to the left ...
1,the bathroom is in the toilet,the toilet is in the bathroom,Paraphrase logically: 'the bathroom is in the ...
2,the woman is to the right of the shirt,the shirt is to the right of the woman,Paraphrase logically: 'the woman is to the rig...
3,the sidewalk is near the street,the street is near the sidewalk,Paraphrase logically: 'the sidewalk is near th...
4,the road is parked on the bus,the bus is parked on the road,Paraphrase logically: 'the road is parked on t...
...,...,...,...
19144,the shirt is to the left of the laptop,the laptop is to the left of the shirt,Paraphrase logically: 'the shirt is to the lef...
19145,the road is traveling down the car,the car is traveling down the road,Paraphrase logically: 'the road is traveling d...
19146,the building is to the left of the roof,the roof is to the left of the building,Paraphrase logically: 'the building is to the ...
19147,the platform is to the left of the giraffe,the giraffe is to the left of the platform,Paraphrase logically: 'the platform is to the ...


In [5]:
data.rename(columns={'wrong_text': 'rejected_response', 'correct_text': 'chosen_response', 'prompt': 'instruction'}, inplace=True)

In [6]:
from datasets import Dataset

dataset = Dataset.from_pandas(data)

In [16]:
import nltk
from nltk.corpus import wordnet as wn

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def is_noun(tag):
    return tag in ['NN', 'NNS', 'NNP', 'NNPS']

def extract_nouns(sentence):
    tokens = nltk.word_tokenize(sentence)
    tagged = nltk.pos_tag(tokens)
    return [word.lower() for word, tag in tagged if is_noun(tag)]

def calculate_similarity(str1, str2):
    nouns1 = extract_nouns(str1)
    nouns2 = extract_nouns(str2)

    if not nouns1 or not nouns2:
        return 0.0

    common_nouns_count = 0

    for noun in nouns1:
        if noun in nouns2:
            common_nouns_count += 1

    similarity_score = common_nouns_count / max(len(nouns1), len(nouns2))

    return similarity_score

# Пример использования функции
string1 = "The quick brown fox jumps over the lazy dog"
string2 = "A fast red fox leaps over a sleepy canine fox fox fox fox fox"
similarity = calculate_similarity(string1, string2)
print(f"Коэффициент схожести: {similarity}")


Коэффициент схожести: 0.14285714285714285


[nltk_data] Downloading package punkt to /home/zoya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zoya/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/zoya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)

model_name = "distilroberta-base"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weigh

In [8]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

In [9]:

def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}

    # Prepend the prompt and a line break to the original_response and response-1 fields.
    prompt_plus_chosen_response = examples["instruction"] + "\n" + examples["chosen_response"]
    prompt_plus_rejected_response = examples["instruction"] + "\n" + examples["rejected_response"]

    # Then tokenize these modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }
    
formatted_dataset = dataset.map(formatting_func) 
formatted_dataset = formatted_dataset.train_test_split()


training_args = TrainingArguments(
    output_dir="./reward_model",
    per_device_train_batch_size=1,
    evaluation_strategy="steps",  
    logging_steps=200,  
)

trainer = RewardTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset["train"],
    eval_dataset=formatted_dataset["test"],
)

trainer.train()

  0%|          | 0/19149 [00:00<?, ?ex/s]

/home/zoya/trl/trl/trainer/reward_trainer.py:128: UserWarning: When using RewardDataCollatorWithPadding, you should set `max_length` in the RewardTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/home/zoya/trl/trl/trainer/reward_trainer.py:139: UserWarning: When using RewardDataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/home/zoya/.local/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: babycar27 (cds). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/zoya/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Accuracy
200,0.251100,0.001807,0.999582
400,0.016300,0.004264,0.999582
600,0.053700,0.004030,0.998747
800,0.012500,0.001238,0.999373
1000,0.013700,0.001605,0.999373


/home/zoya/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/zoya/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2395: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


KeyboardInterrupt: 

In [29]:
import torch


tokenizer = AutoTokenizer.from_pretrained("./reward_model/checkpoint-1000")

model = AutoModelForSequenceClassification.from_pretrained("./reward_model/checkpoint-1000") 


def get_score(model, tokenizer, prompt, response):
    # Tokenize the input sequences
    inputs = tokenizer.encode_plus(prompt, response, truncation=True, padding="max_length", max_length=512, return_tensors="pt")

    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the logits
    logits = outputs.logits

    return logits.item()

# Example usage


prompt = "Paraphrase logically:  table on a book"
example_less_pref_response = "table on a book"
example_preferred_response = "the book on the table	"

score = get_score(model.cpu(), tokenizer, prompt, example_less_pref_response)
print(score)
# >> -3.915163993835449

score = get_score(model.cpu(), tokenizer, prompt, example_preferred_response)
print(score)
# >> 7.460323333740234

-4.245254039764404
7.125707149505615
